<h5>Andrew James<br>Flight Delays Capstone 2<h5>December 2023<br>

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
df = pd.read_csv('/Users/drew/Downloads/flight_delay/2018.csv')

### Data Cleansing 

In [4]:
# df['CANCELLED'] = df['CANCELLED'].round(-1)

The key challenges we faced were centered around:
<li> Inconsistent data types </li> - Some numerical Columns were actually classed as objects...
<li> Renaming ..</li> Completely removing NaN values beyond using df.dropna..

<h3>Dictionary<h3>

FL_DATE<br>
Date of the flight, yy/mm/dd<br>
OP_CARRIER<br>
Airline Identifier<br>
OP_CARRIER_FL_NUM<br>
Flight Number<br>
ORIGIN<br>
Starting Airport Code<br>
DEST<br>
Destination Airport Code<br>
CRS_DEP_TIME<br>
Planned Departure Time<br>
DEP_TIME<br>
Actual Departure Time<br>
DEP_DELAY<br>
Total Delay on Departure in minutes<br>
TAXI_OUT<br>
The time duration elapsed between departure from the origin airport gate and wheels off<br>
WHEELS_OFF<br>
The time point that the aircraft's wheels leave the ground<br>
WHEELS_ON<br>
The time point that the aircraft's wheels touch on the ground<br>
TAXI_IN<br>
The time duration elapsed between wheels-on and gate arrival at the destination airport<br>
CRS_ARR_TIME<br>
Planned arrival time<br>
ARR_TIME<br>
Actual Arrival Time<br>
ARR_DELAY<br>
Total Delay on Arrival in minutes<br>
CANCELLED<br>
Flight Cancelled (1 = cancelled)<br>
CANCELLATION_CODE<br>
Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security<br>
DIVERTED<br>
Aircraft landed on airport that out of schedule<br>
CRS_ELAPSED_TIME<br>
Planned time amount needed for the flight trip<br>
ACTUAL_ELAPSED_TIME<br>
AIR_TIME+TAXI_IN+TAXI_OUT<br>
AIR_TIME<br>
The time duration between wheels_off and wheels_on time<br>
DISTANCE<br>
Distance between two airports<br>
CARRIER_DELAY<br>
Delay caused by the airline in minutes<br>
WEATHER_DELAY<br>
Delay caused by weather<br>
NAS_DELAY<br>
Delay caused by air system<br>
LATE_AIRCRAFT_DELAY<br>
Delay caused by security<br>


<h2>Defining DataFrames <h2>

In [ ]:
# non_cdf is the main df with NO cancelled flights
non_cdf = df[df['CANCELLED']==0]
nan_clmns = ['CANCELLATION_CODE','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']
non_cdf.drop(columns=nan_clmns, inplace=True)
nan_test = non_cdf.isna()
any_nan = nan_test.any().any()
# print(f"cdf contains nan values... {any_nan}")

In [ ]:
# cl_df is the main df with dropped NaN values from the listed columns below
delay_colums = ['CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
cl_df = df.dropna(subset=delay_colums)

In [ ]:
# cdf is the main df with ONLY cancelled flights
cdf = df[df['CANCELLED']==1]

<h3>Taking a micro view of Airports and Airlines based on volume...<h3>

Upon examining the dataset, a need to focus on the busiest airports and airlines was needed to remove outliers. For airports, the 90th percentile in flight volume was taken. The 50th percentile in flight volume of airlines was taken due to fewer airlines than the number of airports in the dataset.<br> ***From this point forward, all airline/airport data is referencing a modified dataframe.

In [ ]:
# 90th,10th percentiles Airports based on airport traffic
departures = df['ORIGIN'].value_counts()
arrivals = df['DEST'].value_counts()
total_flights_airport = departures  + arrivals

airport_percentile_90 = np.percentile(total_flights_airport,90)
airport_percentile_10 = np.percentile(total_flights_airport,10)

# Defining Top Airports
airports_90 = total_flights_airport[total_flights_airport >= airport_percentile_90]
airports_10 = total_flights_airport[total_flights_airport <= airport_percentile_10]

top_airports_list = airports_90.index.tolist()
top_airports = df[df['ORIGIN'].isin(top_airports_list)] # filtering for only the busiest airports in the 90th percentile.




# Top 50th percentile was used for Airlines based on volume due to the limited number of airlines. 
total_flights_airline = df['OP_CARRIER'].value_counts()

airline_percentile_50 = np.percentile(total_flights_airline,50)

# Defining Top Airlines
airline_50t = total_flights_airline[total_flights_airline >= airline_percentile_50]
airline_50b = total_flights_airline[total_flights_airline <= airline_percentile_50]

top_airline_list = airline_50t.index.tolist()
top_airlines = df[df['OP_CARRIER'].isin(top_airline_list)]


# Both top_airports and top_airlines include each other i.e top_airports only filters the 90th% of airports but includes all airlines.

 <h1>Summary <h1>
    

Flight Travel is synonymous with modern business today.
As a flights Aviation Analyst, we seek to understand why airlines might have delays and whether this is man-made or subject to avoidance.

In our below presentation, we focus on analyzing the following core themes:

<li> Flight Volume </li>
<li> Frequency of Cancellations Globally </li>
<li> Duration of Delays </li>



The following analysis was conducted on 7,213,446 flights in 2018. Of those flights, 116,584 were cancelled. The question of why were these flights cancelled is discussed below. July is the busiest month of the year. Mondays are the busiest day of the week. Atlanta is the busiest airport in the US, while Southwest Airlines operates the most flights. 1.61% of flights are cancelled, with weather being the leading cause of cancellation. 33% of delays occur due to disruption in the air system. As a result, planes become stagnant on the runway. Taxi-Out times are statistically significant with respect to departure delays. <br><br> The recommendations to airline companies and airport are as followed: Expect weather delays in months January and March. A primary operations bottleneck is the Taxi-Out time for an aircraft due to delays in airspace. 

 <h3>Volume Insights <h3>
    

In [ ]:
# cdf = cdf[['FL_DATE','OP_CARRIER_FL_NUM','ORIGIN','DEST','CRS_DEP_TIME','DISTANCE','DAY','Month']]

In [ ]:
# Industry Average Cancelled Flights (%)
perc_cancelled = round(df['CANCELLED'].mean()*100,2)
print(f"{perc_cancelled}% of flights cancelled annually")

In [ ]:
df['ORIGIN'].mean()

In [ ]:
# subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5 ))

# Number of flights for each airport
departures = df['ORIGIN'].value_counts()
arrivals = df['DEST'].value_counts()
total_flights_airport = departures  + arrivals

top_airports = pd.DataFrame({'TotalFlights': total_flights_airport})
top_airports.sort_values(by='TotalFlights', ascending=False, inplace=True)
top_airports_20 = top_airports.head(20)


top_airports_20.plot(kind='bar', ax=axes[0])
axes[0].set_ylabel("Annual Flights")
axes[0].set_title("Total Flights by Airport")
labels1 = ['0', '100,000', '200,000', '300,000', '400,000', '500,000','600,000','700,000', '800,000']
axes[0].set_yticks([0,100000,200000,300000,400000,500000,600000, 700000, 800000], labels1)

# Number of flights for each airline

flights = df['OP_CARRIER'].value_counts()
flights.sort_values(ascending= False).head(20).plot(kind='bar', ax=axes[1])
labels = ['0', '200,000', '400,000', '600,000', '800,000', '1,000,000','1,200,000','1,400,000']
axes[1].set_yticks([0,200000,400000,600000,800000,1000000,1200000,1400000], labels)
axes[1].set_ylabel("Annual Flights")
axes[1].set_title("Total Flights by Airline")

plt.tight_layout(pad=5)
plt.show()

In [ ]:
# subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Flights by month Industry Total
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
# adding month column to df
df['Month'] = df['FL_DATE'].dt.month_name()

month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']
df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)
month_count = df['Month'].value_counts().sort_index()
month_count.plot(kind='bar', ax=axes[0])

axes[0].set_ylim(500000, month_count.max() + 10000)
axes[0].set_xlabel("Month")
axes[0].set_title("Flights per Month")



# Flights by Day Industry Total
# Adding Day column to df
df['DAY'] = df['FL_DATE'].dt.day_name()
# df['DAY'] = df['FL_DATE'].dt.weekday
day_count = df['DAY'].value_counts().sort_index()
# day_count
day_count = day_count.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday'])
day_count.plot(kind='bar', ax=axes[1])

axes[1].set_ylim(800000, day_count.max()+1000)
axes[1].set_labels2 = ['800,000','900,000','1,000,000','1,100,000','1,200,000']
axes[1].set_yticks([800000,900000,1000000,1100000,1200000], labels2)
axes[1].set_title("Flights per Day")

plt.tight_layout(pad=5)
plt.show()

<h3> Beyond understanding the volume of flights... </h3>

<h3> Flight Delays  <h3><h4>The industry average for Departure delays is 9.9 minutes and Arrivals are 5.0 minutes delayed.<br><br>The top five airlines have an average Departure delay between 7 - 11 minutes. <br>Delta (DL) Arrives 0.2 minutes early on average while Skywest (OO) averages 7 minutes late.  <h4>

In [ ]:
# subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Average length of delay for Departures in (min)
dep_delay = top_airlines.groupby('OP_CARRIER')['DEP_DELAY'].mean().sort_values(ascending = False)
dep_delay.plot(kind='bar', ax=axes[0])           

sl = 9.96
axes[0].axhline(y=sl, linestyle='--', color='orange', label='Industry_Avg' )
axes[0].set_title("Departure Delay by Airline")
axes[0].set_xlabel('Airline')
axes[0].set_ylabel('Delay (min)')


# Average length of delay for Arrivals in (min)
arr_delay = top_airlines.groupby('OP_CARRIER')['ARR_DELAY'].mean().sort_values(ascending = False)
arr_delay.plot(kind='bar', ax=axes[1])


sl1 = 5.04
axes[1].axhline(y=sl1, linestyle='--', color='orange', label='Industry_Avg' )
axes[1].set_title("Arrival Delay by Airline")
axes[1].set_xlabel('Airline')
axes[1].set_ylabel('Delay (min)')

plt.tight_layout(pad=5)
plt.show()


In [ ]:
# delay percent of total
delay_columns = ['CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
delay_df = cl_df[cl_df[delay_columns] > 0]
delay_count = delay_df[delay_columns].count()
delay_total = delay_df[delay_columns].count().sum()
percent_delay = (delay_count / delay_total)*100
# percent_delay
plt.pie(percent_delay, labels=delay_columns, autopct='%1.1f%%')
plt.title("Type of Delay")
plt.show()


<h4>The average delay duration in minutes for each of the following: <br><br>Carrier 19.4<br>Late Aircraft 25.6<br> National Air System 15.8<br>Weather 3.6                                              <h4>

<h3>Taxi times by airport and airline <h3>

Industry Average Taxi-Out takes 17 minutes, while Taxi-In takes 7 minutes.<br><br>These figures represent the time between when the plane is at the terminal and in the air.<br><br> This could be an indication of prior delays in the airport operations.<br> JFK has the slowest Taxi-Out time of 27 minutes, while MIA has the slowest Taxi-In time of 8.5 minutes. Additional data is needed to determine whether these times are related to the size of the runway.     

In [ ]:
# subplots combined
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Airport Avg Taxi_In times (min)
top_airports.groupby('ORIGIN')['TAXI_IN'].mean().sort_values(ascending=False).head(20).plot(kind='bar', ax=axes[1])
taxi_in_avg = top_airlines['TAXI_IN'].mean()
axes[1].axhline(y=taxi_in_avg, linestyle='--', color='orange', label='Industry_Avg')
axes[1].set_title("Average Taxi_In times (min)")
axes[1].set_ylabel('Minutes')
axes[1].set_ylim(7, 8.75)

# Airport Avg Taxi_Out times (min)
top_airports.groupby('ORIGIN')['TAXI_OUT'].mean().sort_values(ascending=False).head(20).plot(kind='bar', ax=axes[0])
taxi_out_avg = top_airlines['TAXI_OUT'].mean()
axes[0].axhline(y=taxi_out_avg, linestyle='--', color='orange', label='Industry_Avg')
axes[0].set_title("Average Taxi_Out times (min)")
axes[0].set_ylabel('Minutes')

plt.tight_layout(pad=5)
plt.show()


<h3>Flight Cancellations<h3>

<h4> The total flights cancelled in the industry 116,584 (1.6%). <br>The top three airlines with the most cancellations are OH 4.3%, MQ 3.5%, YX 3.2%. <br>January has the most cancellations of any month at 3%, compared to the lowest month October at 0.72%.  <h4>

In [ ]:
# subplots combined
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

cdf = df[df['CANCELLED']==1]
cdf.groupby('OP_CARRIER').sum()['CANCELLED'].sort_values(ascending=False).plot(kind='bar', ax=axes[0])
axes[0].set_title("Annual Cancellations by Airline")
axes[0].set_xlabel('Airline')
axes[0].set_ylabel('Total Cancellations')




# Percent Canceled by airline
canceled_flights = df[df['CANCELLED'] == 1]
canceled_flights
canceled_count = canceled_flights['OP_CARRIER'].value_counts()
canceled_count
total_flights = df['OP_CARRIER'].value_counts()
perc_canceled = canceled_count / total_flights * 100 
perc_canceled.sort_values(ascending= False).head(20).plot(kind='bar', ax=axes[1])

second_line = 1.6
axes[1].axhline(y=second_line, linestyle='--', color='orange', label='Industry_Avg' )

axes[1].set_title("Percent Cancelled by Airline")
axes[1].set_xlabel('Airline')
axes[1].set_ylabel('(%) Cancellations')
plt.show()




In [ ]:
# Average cancelled flights by Airport (%) 



mean_cancelled_airports_origin = top_airports.groupby('ORIGIN').mean(numeric_only=True)['CANCELLED']*100
mean_cancelled_airports_dest = top_airports.groupby('DEST').mean(numeric_only=True)['CANCELLED']*100
mean_cancelled_airports = mean_cancelled_airports_origin + mean_cancelled_airports_dest
mean_cancelled_airports.sort_values(ascending=False).head(20).plot(kind='bar')

second_line = 1.6
plt.axhline(y=second_line, linestyle='--', color='orange', label='Industry_Avg' )

plt.title("Average Cancelled Flights per Year")
plt.ylabel('( % )')
plt.xlabel('Busiest airports in 90th percentile')
plt.show()


In [ ]:
mean_cancelled_carriers = df.groupby('OP_CARRIER').mean()['CANCELLED'].sort_values(ascending=False)*100

<h4> Wednesday has the most cancellations at 1.98% compared to Saturday with the lowest (1.01%)<h4>

In [ ]:
# subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Cancelled flights by month (%)
cdf['FL_DATE'] = pd.to_datetime(cdf['FL_DATE'])
cdf['Month'] = cdf['FL_DATE'].dt.month_name()
cancelled_month_count = cdf['Month'].value_counts()
((cancelled_month_count / month_count)*100).sort_values(ascending=False).plot(kind='bar', ax=axes[0])

# month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']
# cdf['Month'] = pd.Categorical(cdf['Month'], categories=month_order, ordered=True)

sl2 = 1.6
axes[0].axhline(y=second_line, linestyle='--', color='orange', label='Industry_Avg' )
axes[0].set_title("Percent Cancelled Flights per Month")
axes[0].set_xlabel('Month')
axes[0].set_ylabel('( % )')



# Cancelled flights by day (%)

# defining Day column
cdf['DAY'] = cdf['FL_DATE'].dt.day_name()
cancelled_day_count = cdf['DAY'].value_counts()
((cancelled_day_count / day_count)*100).sort_values(ascending=False).plot(kind='bar', ax=axes[1])


axes[1].axhline(y=sl2, linestyle='--', color='orange', label='Industry_Avg' )
axes[1].set_title("Percent Cancelled Flights per day")
axes[1].set_xlabel('Day')
axes[1].set_ylabel('( % )')
plt.tight_layout(pad=5.0)
plt.show()


<h4>Weather is the leading cause of flight cancellation for 9/12 months.                             <h4> 

<h3>Cancellations based on time of day<h3>

<h4>Scheduled flight departures were rounded to the nearest hour. 
    8.4% of cancellations occur during the 16:00 hour. Atlanta (ATL), the busiest airport in the US experiences the most cancellations at the 15:00 hour. The busiest airline, Southwest (WN) has the most cancellations at 6:00.<br><br>** 'TIME_ROUND' was calculated by rounding the departure time to the nearest hour. A significant reduction of unique values was achieved to match common times.  <h4>

In [ ]:
# Histogram of flights cancelled time of day
plt.hist(cdf['CRS_DEP_TIME'])
plt.title("Flights Cancelled by time of day")
labels_x = list(range(0, 2401, 100))
plt.xticks(labels_x, rotation = 45)
plt.xlabel('Hours')
plt.show()



<h3>Correlations and Regression <h3>

In [ ]:
delay_clmns = ['CRS_DEP_TIME','DEP_DELAY','CRS_ARR_TIME','ARR_DELAY','TAXI_OUT','TAXI_IN']
sub_df = non_cdf[delay_clmns]
# sub_df
delay_corr = sub_df.corr()
sns.heatmap(delay_corr, annot=True)

<h4>Arrival Delay is correlated with Taxi-Out and Taxi-In times; however, they are considered weak correlations with correlation values >0.22. <h4>

<h3>Multivariate Regression<h3>

In [ ]:
reg_model

<h4> 94% of the model is explained by the regression analysis. Each of the variables are highly significant with P-values equaling 0 despite having weak correlation. This signals that the dependent variable "Depart Delay" is not influenced by random events.<h4>

<h4>After observing weak correlation in the multivariate regression, a logistic regression was performed. Despite the more involved logistic regression, only 3% of the cancelled flights could be explained by the model.  <h4>

<h2> Final Thoughts<h2>

The primary error in both models was 'model imbalance'. This occurred because the target variable accounted for less than 2% of the model. Attempting to reduce the number of instances in the majority class via under sampling proved unsuccessful. Additional models that are more immune to model imbalance such as random forest, decision tree analysis could prove useful at predicting cancelled flights.<br><br> Despite minimal correlation, the dataset highlighted several main themes: <br><br>
<li> Weather has the greatest effect on flight cancellations, especially during January and March</li> 
<li>The time of day with the most cancellations is 16:00</li> 
<li>Air traffic effects taxi times which play a significant role in flight delays </li> 